#### To conduct different experiements, the only thing that needs to be changed is:
- the name of the molecule
- the molecule's path (check out the molecules folder for whats available
- (optional) number of active of active orbitals and electrons
- ansatz (modify the ansatz based on the number of qubits)

### Overview of the VQE Algorithm
The VQE Algorithm is a promising near-term quantum hybrid variational algorithm that is often used to fing the groundstate energy of molecules. This information is necessary for applications in the drug discovery world, such as molecular simulation.

The way this algorithm works is by combining both quantum and classical computation to find the lowest possible eigenvalue of a given matrix, which we call a Hamiltonian. Hamiltonians represent all the kinetic and potential energies of a given system. The eigenvalue we find corresponds to the groundstate energy of a molecule for a given bond length.

The way a quantum computer does this, is by relying on something called the **variational principle**. Formally, this principles states that:

$\lambda_{min} <= \langle\psi| H |\psi\rangle$

meaning that the lowest possible eigenvalue of a given hermitian matrix will always be less than or equal to the expectation value of $H$

In summary, this means that all we have to do is minimize our expectation value, $\langle\psi(\theta)| H | \psi(\theta)\rangle$, and we will get closer and closer to our ground state energy, but never below it.




*The steps of a VQE to approximate the minimum eigenvalue (our groundstate) are as follows:*
1. Create a paramaterized circuit that will encode our trials states (our ansatz).
    - in this example, we use a hardware efficient ansatz
2. Use the quantum computer to calculate our expectation value $\langle\psi(\theta)| H | \psi(\theta)\rangle$ [where $|\psi(\theta)\rangle$ is our trial state]
3. Read the expectation value out to a classical computer, and then use a classical optimization method to update our parameters.
4. Repeat steps 1-3 until we reach a chemically accurate convergence (around 0.00159 Hartrees)



In [1]:
#import packages
import numpy as np
import pennylane as qml
from pennylane import expval, var, device
pi = np.pi

In [10]:
name ='LiH';charge = 0;multiplicity=1;basis= 'sto-3g';geometry = 'molecules/lih.xyz';
h, nr_qubits = qml.qchem.generate_hamiltonian(
    name,
    geometry,
    charge,
    multiplicity,
    basis,
    mapping='jordan_wigner',
    n_active_orbitals=4,
    n_active_electrons=2,
)
print(h, '\n', nr_qubits)

coeffs = h.coeffs
ops = h.ops
print(nr_qubits)

#of_ham = qml.qchem._terms_to_qubit_operator(coeffs, ops)

(-6.707508290391098) [I0]
+ (-0.1184706446518777) [Z0]
+ (0.0003916086112975293) [Y0 Z1 Y2]
+ (0.0003916086112975293) [X0 Z1 X2]
+ (-0.11847064465187764) [Z1]
+ (0.0003916086112975346) [Y1 Z2 Y3]
+ (0.0003916086112975346) [X1 Z2 X3]
+ (-0.26378756012529625) [Z2]
+ (-0.2637875601252963) [Z3]
+ (-0.2799227166898821) [Z4]
+ (-0.27992271668988217) [Z5]
+ (-0.2799227166898822) [Z6]
+ (-0.2799227166898822) [Z7]
+ (0.13106579035624624) [Z0 Z1]
+ (0.009702948635494247) [Y0 Y2]
+ (0.009702948635494247) [X0 X2]
+ (0.009702948635494247) [Z0 Y1 Z2 Y3]
+ (0.009702948635494247) [Z0 X1 Z2 X3]
+ (0.002366478148277553) [Y0 X1 X2 Y3]
+ (-0.002366478148277553) [Y0 Y1 X2 X3]
+ (-0.002366478148277553) [X0 X1 Y2 Y3]
+ (0.002366478148277553) [X0 Y1 Y2 X3]
+ (0.006795526682424357) [Y0 X1 X4 Y5]
+ (-0.006795526682424357) [Y0 Y1 X4 X5]
+ (-0.006795526682424357) [X0 X1 Y4 Y5]
+ (0.006795526682424357) [X0 Y1 Y4 X5]
+ (0.0067955266824243615) [Y0 X1 X6 Y7]
+ (-0.0067955266824243615) [Y0 Y1 X6 X7]
+ (-0.006795526682

In [17]:
dev = qml.device("default.qubit", wires=8)
def ansatz(params, wires=[0,1,2,3,4,5,6,7]):
    qml.BasisState(np.array([1, 1,1,1,0,0, 0, 0]), wires=wires)
    for i in wires:
        qml.RY(params[i], wires=wires[i])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.CNOT(wires=[wires[4], wires[5]])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.CNOT(wires=[wires[3], wires[4]])
    qml.CNOT(wires=[wires[5], wires[6]])
    qml.CNOT(wires=[wires[6], wires[7]])

    for i in wires:
        qml.RY(params[i+nr_qubits], wires=wires[i])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.CNOT(wires=[wires[2], wires[3]])
    qml.CNOT(wires=[wires[4], wires[5]])
    qml.CNOT(wires=[wires[1], wires[2]])
    qml.CNOT(wires=[wires[3], wires[4]])
    qml.CNOT(wires=[wires[5], wires[6]])
    qml.CNOT(wires=[wires[6], wires[7]])


initial_params = np.random.uniform(low=0, high=2*np.pi, size=(nr_qubits * 2))

In [18]:
cost = qml.VQECost(ansatz, h, dev)
cost(initial_params)

-6.94398328662911

In [19]:
import time
max_iterations = 500
step_size = 0.05
conv_tol = 1e-06
opt = qml.GradientDescentOptimizer(stepsize=0.05)

vqe_cost = []


prev_energy = cost(initial_params)


start = time.time()
params = initial_params
for n in range(max_iterations):

    #calculate vanilla gradient descent
    params = opt.step(cost, params)

    #calculate new energy with updates params
    energy = cost(params)
    vqe_cost.append(energy)

    #calculate convergence
    conv = np.abs(energy - prev_energy)

    if n % 50 == 0:
        print('Iteration = ', n,  'Ground-state energy = ', energy, 'Ha',  'Convergence parameter =', conv, 'Ha')

    if conv <= conv_tol:
        print("Groundate found: {:9f}".format(cost(params)))
        break
        

    prev_energy = energy
end = time.time() - start


Iteration =  0 Ground-state energy =  -6.946855078640122 Ha Convergence parameter = 0.0028717920110121753 Ha
Iteration =  50 Ground-state energy =  -7.1243532788692265 Ha Convergence parameter = 0.004149867587398326 Ha
Iteration =  100 Ground-state energy =  -7.318341462154296 Ha Convergence parameter = 0.003161722201575401 Ha
Iteration =  150 Ground-state energy =  -7.429562770893708 Ha Convergence parameter = 0.0014737221575185089 Ha
Iteration =  200 Ground-state energy =  -7.480017787574433 Ha Convergence parameter = 0.0006718498951876484 Ha
Iteration =  250 Ground-state energy =  -7.503536428375407 Ha Convergence parameter = 0.00032315162425344823 Ha
Iteration =  300 Ground-state energy =  -7.515270909040448 Ha Convergence parameter = 0.0001700935537201076 Ha
Iteration =  350 Ground-state energy =  -7.521844591334168 Ha Convergence parameter = 0.00010346578122799599 Ha
Iteration =  400 Ground-state energy =  -7.526215423484377 Ha Convergence parameter = 7.636980476721789e-05 Ha
Ite

In [20]:
print('{:5f}'.format(end))

7561.228734


In [7]:
import openfermion

#of_ham = qml.qchem._terms_to_qubit_operator(h.coeffs, h.ops)
#lin = openfermion.utils.LinearQubitOperator(of_ham, 4)
#ground_state = openfermion.utils.get_ground_state(lin)
#print(ground_state[0])

In [8]:
from matplotlib import pyplot as plt

plt.style.use("seaborn")
fig = plt.figure(figsize=(7,5))

plt.plot(vqe_cost, "b", label="VQE Cost")
#plt.axhline(y=ground_state[0], color='r', linestyle='--', label='Exact groundstate')

plt.ylabel("Energy", fontsize=18)
plt.xlabel("Step", fontsize=18)
plt.title('H2 Groundstate Energy', fontsize = 18)
#plt.axhline(y = -1.136189454088, label='exact groundstate', color='red')


plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

plt.legend(fontsize=18)
plt.show()

<Figure size 700x500 with 1 Axes>